In [ ]:
import pandas as pd
import folium
from folium.plugins import HeatMap, MarkerCluster

# CSV 데이터 경로 설정
file_path = r"C:\Users\jiyon\Documents\hello\Piuda_project\accident\accident_2022.csv"

# 데이터 로드 및 전처리
df = pd.read_csv(file_path)
df.rename(columns={
    '사건명': 'incident_name',
    '사고발생일시': 'accident_datetime',
    '사고종류': 'accident_type',
    '안전사고유형': 'safety_incident_type',
    '사망자': 'deaths',
    '실종자': 'missing',
    '사망실종자': 'deaths_and_missing',
    '부상자': 'injuries',
    '해역': 'region',
    '선박용도(통계)': 'vessel_usage_stat',
    '선박용도(대)': 'vessel_usage_major',
    '선박용도(중)': 'vessel_usage_mid',
    '선박용도(소)': 'vessel_usage_minor',
    '선박톤수': 'vessel_tonnage',
    '톤수(통계)': 'tonnage_stat',
    '위도': 'latitude',
    '경도': 'longitude'
}, inplace=True)

# datetime 컬럼 변환
df['accident_datetime'] = pd.to_datetime(df['accident_datetime'])

# 지도 시각화 함수
def visualize_accident_data(df):
    # 지도 초기화 (한국 중심 좌표)
    accident_map = folium.Map(location=[36.5, 127.5], zoom_start=7)

    # 사고 클러스터 추가
    marker_cluster = MarkerCluster().add_to(accident_map)

    # 사고 유형별 마커 색상 정의
    type_colors = {
        "침몰": "red",
        "좌초": "blue",
        "충돌": "green",
        "운항저해": "purple",
        "기타": "orange"
    }

    # 마커 추가 부분 수정
    for _, row in df.iterrows():
        location = [row['latitude'], row['longitude']]
        
        # 스타일 추가 (글자 크기 조정)
        popup_content = f"""
        <div style="font-size: 15px;">
            <b>사건명:</b> {row['incident_name']}<br>
            <b>사고발생일시:</b> {row['accident_datetime']}<br>
            <b>사고종류:</b> {row['accident_type']}<br>
            <b>안전사고유형:</b> {row['safety_incident_type'] if row['safety_incident_type'] else "N/A"}<br>
            <b>사망자:</b> {row['deaths']}<br>
            <b>실종자:</b> {row['missing']}<br>
            <b>부상자:</b> {row['injuries']}<br>
            <b>해역:</b> {row['region']}<br>
            <b>선박용도(통계):</b> {row['vessel_usage_stat']}<br>
            <b>선박톤수:</b> {row['vessel_tonnage']} 톤
        </div>
        """
        
        folium.Marker(
            location,
            popup=folium.Popup(popup_content, max_width=300),
            icon=folium.Icon(color=type_colors.get(row['accident_type'], "gray"))
        ).add_to(marker_cluster)

    # Heatmap 추가
    heat_data = [[row['latitude'], row['longitude']] for _, row in df.iterrows()]

    # HeatMap에 가중치 및 설정 추가
    HeatMap(
        heat_data,
        radius=15,  # 기본 반경 크기 (값을 낮추면 히트맵이 더 날카로워짐)
        blur=10,    # 블러링 크기 (값을 낮추면 디테일이 더 선명해짐)
        max_zoom=10, # 히트맵이 적용되는 최대 줌 단계
        min_opacity=0.2  # 히트맵의 최소 불투명도 (확대/축소 시 대비 효과)
    ).add_to(accident_map)

    # HTML로 저장
    accident_map.save('accident_visualization_2022.html')
    print("파일 생성")

# 사고 데이터 지도 시각화 실행
visualize_accident_data(df)
